In [3]:
import numpy as np
import pandas as pd
import unidecode as u
import geonamescache as g

df = pd.read_pickle("./data.pkl")
df = df.dropna(thresh=2).sort_values(by=['city'])

def addCleanName(city):
    uc = u.unidecode(city['name'])
    firstBracket = uc.find('(')
    if firstBracket > 0:
        uc = uc[:firstBracket]
    city['cleanName'] = uc.strip()
    return city

gc = g.GeonamesCache()
countries = gc.get_countries()
cities = {k: addCleanName(c) for k, c in gc.get_cities().items()}

groupedCities = {}
for k, v in cities.items():
    cn = v['cleanName']
    if cn in groupedCities:
        cg = groupedCities[cn]
    else:
        cg = []
    groupedCities[cn] = cg + [v]

def largestCityByName(cleanCity, country):
    if type(country) == type({}):
        cities = [c for c in groupedCities[cleanCity] if c['countrycode'] == country['iso']]
    else:
        cities = groupedCities[cleanCity]
    city = cities[0]
    for c in cities[1:]:
        if c['population'] > city['population']:
            city = c
    return city

matchedCities = [largestCityByName(d['city'], countries.get(d['country'])) for d in df[['city','country']].to_dict('records')]
df['latitude'] = [c['latitude'] for c in matchedCities]
df['longitude'] = [c['longitude'] for c in matchedCities]
df['countrycode'] = [c['countrycode'] for c in matchedCities]

df=df.drop('country', axis=1)

df.to_pickle('./data2.pkl')
